In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define Franke Function
def FrankeFunction(x, y):
    term1 = 0.75 * np.exp(-(0.25 * (9 * x - 2) ** 2) - 0.25 * ((9 * y - 2) ** 2))
    term2 = 0.75 * np.exp(-((9 * x + 1) ** 2) / 49.0 - 0.1 * (9 * y + 1))
    term3 = 0.5 * np.exp(-(9 * x - 7) ** 2 / 4.0 - 0.25 * ((9 * y - 3) ** 2))
    term4 = -0.2 * np.exp(-(9 * x - 4) ** 2 - (9 * y - 7) ** 2)
    return term1 + term2 + term3 + term4

def create_X(x, y, n):
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n + 1) * (n + 2) / 2)
    X = np.ones((N, l))

    for i in range(1, n + 1):
        q = int((i) * (i + 1) / 2)
        for k in range(i + 1):
            X[:, q + k] = (x ** (i - k)) * (y ** k)

    return X

# Making meshgrid of datapoints and compute Franke's function
n = 5
N = 1000
x = np.sort(np.random.uniform(0, 1, N))
y = np.sort(np.random.uniform(0, 1, N))
z = FrankeFunction(x, y)
mse_train = np.zeros(n)
r2_train = np.zeros(n)
beta_val = np.zeros(n)

mse_test = np.zeros(n)
r2_test = np.zeros(n)

beta = []

# Base implementation of OLS
for degree in range(0, n):
    # Create the Design matrix X with polynomial degree = degree
    X = create_X(x, y, degree)
    # split in training and test data
    X_train, X_test, y_train, y_test = train_test_split(X, z, test_size=0.2)

    # Scaling the data
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Calculating the beta parameter using the standard OLS formula (X^TX)^-1X^Ty
    beta.append((np.linalg.pinv(X_train_scaled.T @ X_train_scaled) @ X_train_scaled.T) @ y_train)

    # Predicting the values using the formula X * Beta for test and train data
    y_tilde_train = X_train_scaled @ beta[degree]
    y_tilde_test = X_test_scaled @ beta[degree]

    # Calculating MSE and R2 Score for train prediction
    mse_train[degree] = np.mean(np.mean((y_train - y_tilde_train) ** 2))
    r2_train[degree] = r2_score(y_train, y_tilde_train)

    # Calculating MSE and R2 Score for test prediction
    mse_test[degree] = np.mean(np.mean((y_test - y_tilde_test) ** 2))
    r2_test[degree] = r2_score(y_test, y_tilde_test)

# Transpose the beta list for table plotting
beta = np.array(beta).T.tolist()

# Plotting the beta values as a table
plt.figure(figsize=(10, 6))
plt.title('Beta Values')
plt.axis('off')  # Turn off axis
plt.table(cellText=beta, rowLabels=[f'Degree {i}' for i in range(n)], loc='center')
plt.show()


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.